In [74]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

In [75]:
from kafka import KafkaConsumer
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, StringType
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import to_timestamp

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("page_id", IntegerType(), True),
    StructField("url", StringType(), True),
    StructField("head", StringType(), True),
    StructField("author", StringType(), True),
    StructField("category", StringType(), True),
    StructField("date", StringType(), True),
    StructField("tags", StringType(), True),
    StructField("text", StringType(), True),
    StructField("summary", StringType(), True),
])

# offset: subscribe from beginning of time
consumer = KafkaConsumer('news_from_crawler', bootstrap_servers='127.0.0.1:9092', auto_offset_reset='earliest')


In [76]:
rows = []
while True:
    partition = ""
    for _, partition in consumer.poll(timeout_ms=5000, max_records=999999).items():
        for msg in partition:
            data = msg.value.decode('utf-8').split('\t')
            rows.append({
                "id": int(data[0]),
                "page_id": int(data[1]),
                "url": data[2],
                "head": data[3],
                "author": data[4],
                "category": data[5],
                "date": data[6],
                "tags": data[7],
                "text": data[8],
                "summary": data[9],
            })
            print(f'\r{len(rows)}', end='')
    if len(partition) == 0:
        break

# to df
df = pd.DataFrame(rows)


1597

In [77]:
import pandas as pd
# csv_file_path = 'news_202307170512.csv'
# df = pd.read_csv(csv_file_path, delimiter='\t')
print(df)

         id   page_id                                                url   
0     32298   2172793  https://www.tgju.org/news/2630723/سناریوی-کاهش...  \
1         3        19             https://www.tgju.org/news/2638096/2-49   
2         4        21  https://www.tgju.org/news/2638081/چشم-انداز-بی...   
3         5        23  https://www.tgju.org/news/2638059/آیا-اکنون-وق...   
4        30        50  https://www.tgju.org/news/2638164/تحلیل-و-پیش-...   
...     ...       ...                                                ...   
1592  39106   1296774  https://www.tgju.org/news/2627237/آیا-طلا-دوبا...   
1593  39109   2172897  https://www.tgju.org/news/2567136/آنجا-که-همه-...   
1594  39113  10078821  https://www.tgju.org/news/2391525/از-هر-سه-زن-...   
1595  39137  10990361  https://www.tgju.org/news/2579666/پیش-بینی-واق...   
1596  39142  10990255  https://www.tgju.org/news/2617083/آیا-طلا-تا-پ...   

                                               head            author   
0             

In [78]:
df

,id,page_id,url,head,author,category,date,tags,text,summary
0,32298,2172793,https://www.tgju.org/news/2630723/سناریوی-کاهش...,سناریوی کاهش نفت خام، ادامه‌دار,معصومه دانش,اخبار طلا و سکه,دوشنبه 22 خرداد 1402,بهای نفت|نفت خام|نفت برنت|نرخ بهره|افزایش نرخ ...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از رو...,از آنجا که نگرانی در مورد موضوعاتی همچون تقاضا...
1,3,19,https://www.tgju.org/news/2638096/2-49,خرس‌های طلا در کمین!,معصومه دانش,اخبار طلا و سکه,یکشنبه 25 تیر 1402,بهای طلا|بهای نقره|بازار طلا|نرخ بهره|افزایش ن...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از «ک...,گرچه طلا قیمت‌های طلا در هفته‌های اخیر رشد داش...
2,4,21,https://www.tgju.org/news/2638081/چشم-انداز-بی...,چشم‌انداز بیت کوین تا پایان ماه جولای,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,ریپل|بیت کوین|اتریوم|ارزهای دیجیتال|اصلاح قیمت...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از بی...,قیمت بیت‌کوین تحت تاثیر تحولات اخیر در پرونده‌...
3,5,23,https://www.tgju.org/news/2638059/آیا-اکنون-وق...,آیا اکنون وقت خرید ریپل است؟,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,ریپل|کمیسیون بورس و اوراق بهادار آمریکا|دادگاه...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از کو...,با انتشار خبری مبنی بر صدور یک حکم مطلوب برای ...
4,30,50,https://www.tgju.org/news/2638164/تحلیل-و-پیش-...,تحلیل و پیش‌بینی قیمت ریپل؛ 25 تیر,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,بیت‌کوین|ریپل|ارزهای دیجیتال|بلک راک|فیدلیتی|ص...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از کو...,پس از پیروزی نسبی ریپل در دادگاه،‌ قیمت آن به ...
...,...,...,...,...,...,...,...,...,...,...
1592,39106,1296774,https://www.tgju.org/news/2627237/آیا-طلا-دوبا...,آیا طلا دوباره به 1900 دلار می‌رسد؟,معصومه دانش,اخبار طلا و سکه,یکشنبه 7 خرداد 1402,بهای طلا|خریداران طلا|دلار آمریکا|نرخ بهره|افز...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از «ا...,در بحبوحه‌ی انتظارات برای افزایش 25 واحد پایه‌...
1593,39109,2172897,https://www.tgju.org/news/2567136/آنجا-که-همه-...,آنجا که همه طلا بخواهند!,معصومه دانش,اخبار طلا و سکه,چهارشنبه 25 آبان 1401,بحران روسیه و اوکراین|معاملات آتی طلا|دارایی ا...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از ای...,با بروز نگرانی در مورد رویداد انفجار موشکی در ...
1594,39113,10078821,https://www.tgju.org/news/2391525/از-هر-سه-زن-...,از هر سه زن، یک نفر قصد خرید ارز دیجیتال دارد,گروه تولید محتوا,اخبار ارزهای دیجیتال,جمعه 13 اسفند 1400,زنان|ارزدیجیتال|تحلیل بنیادین|اخبار ویژه,به گزارش شبکه اطلاع رسانی طلا، سکه و ارز و به ...,بلاک‌فای با آماری که از مقاله مروری خود گرفت ا...
1595,39137,10990361,https://www.tgju.org/news/2579666/پیش-بینی-واق...,پیش‌بینی واقع‌بینانه‌ی قیمت طلا,معصومه دانش,اخبار طلا و سکه,شنبه 26 آذر 1401,فلزات گرانبها|گزارش تورم|فدرال رزرو|چشم‌انداز ...,به گزارششبکه اطلاع‌رسانی طلا و ارز، سایت «اف‌ا...,از آنجا که سرمایه‌گذاران به طور پیوسته در حال ...


In [79]:
#show full text randomly
pd.set_option('display.max_columns', None)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

desired_field = df.at[60, 'text']
print(desired_field)


به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از کوین ادیشن، بر اساس داده های وب‌سایت ردیاب بازار کوین مارکت کپ، ارزش جهانی بازارارزهای دیجیتالطی 24 ساعت گذشته اندکی افزایش یافته است. در لحظه‌ی نگارش این مطلب، ارزش کل بازار ارزهای دیجیتال با 0.02 درصد افزایش به 824.17 میلیارد دلار رسیده است. نرخ تسلطبیت کوین (BTC)بر بازار کریپتو نیز طی 24 ساعت گذشته 0.02 درصد افزایش یافته است و اکنون به 39.60 درصد رسیده است.موسسه‌ی تحلیل بلاک‌چین سانتیمنت، امروز در توییتر اعلام کرد که ارزش بازار تمام ارزهای دیجیتال موجود در بازار کریپتو «در جهات مختلف و مستقل از یکدیگر کاهش یا افزایش می‌یابند.»موسسه‌ی سانتیمنت در این پست تاکید کرد که در حال حاضر احساسات عموم سرمایه‌گذاران و معامله‌گران نسبت به دو التکوینریپل (XRP)واتریوم (ETH)مثبت و صعودی شده است؛ از سویی دیگر، علاقه‌ی آن‌ها نسبت به بیت کوین (BTC)،بایننس کوین (BNB)وکاردانو (ADA)کاهش یافته است. موسسه‌ی سانتیمنت در پایان این پست خاطرنشان کرد که از نظر تاریخی، پروژه‌هایی که احساسات عمومی بازار نسبت به آن‌ها نزولی است، معمولا به طور متوسط عملکرد بهتری از خود به

In [80]:
desired_field = df.at[60, 'tags']
print(desired_field)

اتریوم|ریپل|بیت‌کوین|بایننس کوین|کاردانو|احساسات بازار|ارزهای دیجیتال|بازار کریپتو|سطح مقاومت|میانگین متحرک نمایی|اخبار ویژه|خبر دو


In [81]:
import re
hashtag_pattern = r"#\w+"

hashtags_list = re.findall(hashtag_pattern, df.at[60,'text'])

dfhashtags = pd.DataFrame(hashtags_list, columns=['text'])

print(dfhashtags)

Empty DataFrame
Columns: [text]
Index: []


In [82]:
desired_field = df.at[60, 'tags']
print(desired_field)

اتریوم|ریپل|بیت‌کوین|بایننس کوین|کاردانو|احساسات بازار|ارزهای دیجیتال|بازار کریپتو|سطح مقاومت|میانگین متحرک نمایی|اخبار ویژه|خبر دو


In [83]:
description = df.describe()
print(description)

                 id       page_id
count   1597.000000  1.597000e+03
mean   30476.777708  3.890506e+06
std    10511.930714  4.431693e+06
min        2.000000  1.800000e+01
25%    31894.000000  5.726550e+05
50%    34596.000000  2.027142e+06
75%    36832.000000  5.852681e+06
max    39333.000000  1.693962e+07


In [84]:
non_numeric_columns = df.select_dtypes(exclude=[np.number]).columns
print("ستون‌های غیر عددی:")
print(non_numeric_columns)

ستون‌های غیر عددی:
Index(['url', 'head', 'author', 'category', 'date', 'tags', 'text', 'summary'], dtype='object')


In [85]:
null_values = df.isnull().sum()
print(null_values)

id          0
page_id     0
url         0
head        0
author      0
category    0
date        0
tags        0
text        0
summary     0
dtype: int64


In [86]:
special_characters = ['?', '!', 'nan','Unknown','Invalid','NAN','Nan','None','#']
for column in df.columns:
    counts = {}
    for char in special_characters:
        counts[char] = df[column].apply(lambda x: str(x).count(char)).sum()
    print(f"ستون {column}: {counts}")

ستون id: {'?': 0, '!': 0, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}
ستون page_id: {'?': 0, '!': 0, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}
ستون url: {'?': 0, '!': 0, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}
ستون head: {'?': 0, '!': 156, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}
ستون author: {'?': 0, '!': 0, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}
ستون category: {'?': 0, '!': 0, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}
ستون date: {'?': 0, '!': 0, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}
ستون tags: {'?': 0, '!': 0, 'nan': 4, 'Unknown': 0, 'Invalid': 0, 'NAN': 1, 'Nan': 0, 'None': 0, '#': 0}
ستون text: {'?': 0, '!': 128, 'nan': 26, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 2, 'None': 0, '#': 21}
ستون summary: {'?': 0, '!': 21, 'nan': 0, '

In [87]:
df = df.dropna(subset=['text', 'category','summary'])


In [88]:
# df.drop('author', axis=1, inplace=True)


In [89]:
special_characters = ['?', '!', 'nan','Unknown','Invalid','NAN','Nan','None','#']
for column in df.columns:
    counts = {}
    for char in special_characters:
        counts[char] = df[column].apply(lambda x: str(x).count(char)).sum()
    print(f"ستون {column}: {counts}")

ستون id: {'?': 0, '!': 0, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}


ستون page_id: {'?': 0, '!': 0, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}
ستون url: {'?': 0, '!': 0, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}
ستون head: {'?': 0, '!': 156, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}
ستون author: {'?': 0, '!': 0, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}
ستون category: {'?': 0, '!': 0, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}
ستون date: {'?': 0, '!': 0, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}
ستون tags: {'?': 0, '!': 0, 'nan': 4, 'Unknown': 0, 'Invalid': 0, 'NAN': 1, 'Nan': 0, 'None': 0, '#': 0}
ستون text: {'?': 0, '!': 128, 'nan': 26, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 2, 'None': 0, '#': 21}
ستون summary: {'?': 0, '!': 21, 'nan': 0, 'Unknown': 0, 'Invalid': 0, 'NAN': 0, 'Nan': 0, 'None': 0, '#': 0}


In [90]:
df

,id,page_id,url,head,author,category,date,tags,text,summary
0,32298,2172793,https://www.tgju.org/news/2630723/سناریوی-کاهش...,سناریوی کاهش نفت خام، ادامه‌دار,معصومه دانش,اخبار طلا و سکه,دوشنبه 22 خرداد 1402,بهای نفت|نفت خام|نفت برنت|نرخ بهره|افزایش نرخ ...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از رو...,از آنجا که نگرانی در مورد موضوعاتی همچون تقاضا...
1,3,19,https://www.tgju.org/news/2638096/2-49,خرس‌های طلا در کمین!,معصومه دانش,اخبار طلا و سکه,یکشنبه 25 تیر 1402,بهای طلا|بهای نقره|بازار طلا|نرخ بهره|افزایش ن...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از «ک...,گرچه طلا قیمت‌های طلا در هفته‌های اخیر رشد داش...
2,4,21,https://www.tgju.org/news/2638081/چشم-انداز-بی...,چشم‌انداز بیت کوین تا پایان ماه جولای,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,ریپل|بیت کوین|اتریوم|ارزهای دیجیتال|اصلاح قیمت...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از بی...,قیمت بیت‌کوین تحت تاثیر تحولات اخیر در پرونده‌...
3,5,23,https://www.tgju.org/news/2638059/آیا-اکنون-وق...,آیا اکنون وقت خرید ریپل است؟,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,ریپل|کمیسیون بورس و اوراق بهادار آمریکا|دادگاه...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از کو...,با انتشار خبری مبنی بر صدور یک حکم مطلوب برای ...
4,30,50,https://www.tgju.org/news/2638164/تحلیل-و-پیش-...,تحلیل و پیش‌بینی قیمت ریپل؛ 25 تیر,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,بیت‌کوین|ریپل|ارزهای دیجیتال|بلک راک|فیدلیتی|ص...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از کو...,پس از پیروزی نسبی ریپل در دادگاه،‌ قیمت آن به ...
...,...,...,...,...,...,...,...,...,...,...
1592,39106,1296774,https://www.tgju.org/news/2627237/آیا-طلا-دوبا...,آیا طلا دوباره به 1900 دلار می‌رسد؟,معصومه دانش,اخبار طلا و سکه,یکشنبه 7 خرداد 1402,بهای طلا|خریداران طلا|دلار آمریکا|نرخ بهره|افز...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از «ا...,در بحبوحه‌ی انتظارات برای افزایش 25 واحد پایه‌...
1593,39109,2172897,https://www.tgju.org/news/2567136/آنجا-که-همه-...,آنجا که همه طلا بخواهند!,معصومه دانش,اخبار طلا و سکه,چهارشنبه 25 آبان 1401,بحران روسیه و اوکراین|معاملات آتی طلا|دارایی ا...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از ای...,با بروز نگرانی در مورد رویداد انفجار موشکی در ...
1594,39113,10078821,https://www.tgju.org/news/2391525/از-هر-سه-زن-...,از هر سه زن، یک نفر قصد خرید ارز دیجیتال دارد,گروه تولید محتوا,اخبار ارزهای دیجیتال,جمعه 13 اسفند 1400,زنان|ارزدیجیتال|تحلیل بنیادین|اخبار ویژه,به گزارش شبکه اطلاع رسانی طلا، سکه و ارز و به ...,بلاک‌فای با آماری که از مقاله مروری خود گرفت ا...
1595,39137,10990361,https://www.tgju.org/news/2579666/پیش-بینی-واق...,پیش‌بینی واقع‌بینانه‌ی قیمت طلا,معصومه دانش,اخبار طلا و سکه,شنبه 26 آذر 1401,فلزات گرانبها|گزارش تورم|فدرال رزرو|چشم‌انداز ...,به گزارششبکه اطلاع‌رسانی طلا و ارز، سایت «اف‌ا...,از آنجا که سرمایه‌گذاران به طور پیوسته در حال ...


In [91]:
import pandas as pd

# Assuming you have already defined the DataFrame 'df' with a 'text' column

# Define the regular expression pattern to match hashtags
hashtag_pattern = r"#\w+"

# Use the str.findall() method to extract hashtags from the 'text' column and create a new column 'hashtags'
df['hashtags'] = df['text'].str.findall(hashtag_pattern)

# Display the DataFrame with the new 'hashtags' column
print(df)

         id   page_id                                                url   
0     32298   2172793  https://www.tgju.org/news/2630723/سناریوی-کاهش...  \
1         3        19             https://www.tgju.org/news/2638096/2-49   
2         4        21  https://www.tgju.org/news/2638081/چشم-انداز-بی...   
3         5        23  https://www.tgju.org/news/2638059/آیا-اکنون-وق...   
4        30        50  https://www.tgju.org/news/2638164/تحلیل-و-پیش-...   
...     ...       ...                                                ...   
1592  39106   1296774  https://www.tgju.org/news/2627237/آیا-طلا-دوبا...   
1593  39109   2172897  https://www.tgju.org/news/2567136/آنجا-که-همه-...   
1594  39113  10078821  https://www.tgju.org/news/2391525/از-هر-سه-زن-...   
1595  39137  10990361  https://www.tgju.org/news/2579666/پیش-بینی-واق...   
1596  39142  10990255  https://www.tgju.org/news/2617083/آیا-طلا-تا-پ...   

                                               head            author   
0             

In [92]:
df

,id,page_id,url,head,author,category,date,tags,text,summary,hashtags
0,32298,2172793,https://www.tgju.org/news/2630723/سناریوی-کاهش...,سناریوی کاهش نفت خام، ادامه‌دار,معصومه دانش,اخبار طلا و سکه,دوشنبه 22 خرداد 1402,بهای نفت|نفت خام|نفت برنت|نرخ بهره|افزایش نرخ ...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از رو...,از آنجا که نگرانی در مورد موضوعاتی همچون تقاضا...,[]
1,3,19,https://www.tgju.org/news/2638096/2-49,خرس‌های طلا در کمین!,معصومه دانش,اخبار طلا و سکه,یکشنبه 25 تیر 1402,بهای طلا|بهای نقره|بازار طلا|نرخ بهره|افزایش ن...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از «ک...,گرچه طلا قیمت‌های طلا در هفته‌های اخیر رشد داش...,[]
2,4,21,https://www.tgju.org/news/2638081/چشم-انداز-بی...,چشم‌انداز بیت کوین تا پایان ماه جولای,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,ریپل|بیت کوین|اتریوم|ارزهای دیجیتال|اصلاح قیمت...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از بی...,قیمت بیت‌کوین تحت تاثیر تحولات اخیر در پرونده‌...,[]
3,5,23,https://www.tgju.org/news/2638059/آیا-اکنون-وق...,آیا اکنون وقت خرید ریپل است؟,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,ریپل|کمیسیون بورس و اوراق بهادار آمریکا|دادگاه...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از کو...,با انتشار خبری مبنی بر صدور یک حکم مطلوب برای ...,[]
4,30,50,https://www.tgju.org/news/2638164/تحلیل-و-پیش-...,تحلیل و پیش‌بینی قیمت ریپل؛ 25 تیر,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,بیت‌کوین|ریپل|ارزهای دیجیتال|بلک راک|فیدلیتی|ص...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از کو...,پس از پیروزی نسبی ریپل در دادگاه،‌ قیمت آن به ...,[]
...,...,...,...,...,...,...,...,...,...,...,...
1592,39106,1296774,https://www.tgju.org/news/2627237/آیا-طلا-دوبا...,آیا طلا دوباره به 1900 دلار می‌رسد؟,معصومه دانش,اخبار طلا و سکه,یکشنبه 7 خرداد 1402,بهای طلا|خریداران طلا|دلار آمریکا|نرخ بهره|افز...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از «ا...,در بحبوحه‌ی انتظارات برای افزایش 25 واحد پایه‌...,[]
1593,39109,2172897,https://www.tgju.org/news/2567136/آنجا-که-همه-...,آنجا که همه طلا بخواهند!,معصومه دانش,اخبار طلا و سکه,چهارشنبه 25 آبان 1401,بحران روسیه و اوکراین|معاملات آتی طلا|دارایی ا...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از ای...,با بروز نگرانی در مورد رویداد انفجار موشکی در ...,[]
1594,39113,10078821,https://www.tgju.org/news/2391525/از-هر-سه-زن-...,از هر سه زن، یک نفر قصد خرید ارز دیجیتال دارد,گروه تولید محتوا,اخبار ارزهای دیجیتال,جمعه 13 اسفند 1400,زنان|ارزدیجیتال|تحلیل بنیادین|اخبار ویژه,به گزارش شبکه اطلاع رسانی طلا، سکه و ارز و به ...,بلاک‌فای با آماری که از مقاله مروری خود گرفت ا...,[]
1595,39137,10990361,https://www.tgju.org/news/2579666/پیش-بینی-واق...,پیش‌بینی واقع‌بینانه‌ی قیمت طلا,معصومه دانش,اخبار طلا و سکه,شنبه 26 آذر 1401,فلزات گرانبها|گزارش تورم|فدرال رزرو|چشم‌انداز ...,به گزارششبکه اطلاع‌رسانی طلا و ارز، سایت «اف‌ا...,از آنجا که سرمایه‌گذاران به طور پیوسته در حال ...,[]


In [93]:
# Use the str.replace() method to remove the '#' symbol from each hashtag in the lists
hashtags_lists = df['hashtags'].apply(lambda hashtags: [tag.replace('#', '') for tag in hashtags])

# Create a new column 'hashtags' in the DataFrame with the cleaned hashtags
df['hashtags'] = hashtags_lists

# Display the DataFrame with the new 'hashtags' column
df

,id,page_id,url,head,author,category,date,tags,text,summary,hashtags
0,32298,2172793,https://www.tgju.org/news/2630723/سناریوی-کاهش...,سناریوی کاهش نفت خام، ادامه‌دار,معصومه دانش,اخبار طلا و سکه,دوشنبه 22 خرداد 1402,بهای نفت|نفت خام|نفت برنت|نرخ بهره|افزایش نرخ ...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از رو...,از آنجا که نگرانی در مورد موضوعاتی همچون تقاضا...,[]
1,3,19,https://www.tgju.org/news/2638096/2-49,خرس‌های طلا در کمین!,معصومه دانش,اخبار طلا و سکه,یکشنبه 25 تیر 1402,بهای طلا|بهای نقره|بازار طلا|نرخ بهره|افزایش ن...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از «ک...,گرچه طلا قیمت‌های طلا در هفته‌های اخیر رشد داش...,[]
2,4,21,https://www.tgju.org/news/2638081/چشم-انداز-بی...,چشم‌انداز بیت کوین تا پایان ماه جولای,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,ریپل|بیت کوین|اتریوم|ارزهای دیجیتال|اصلاح قیمت...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از بی...,قیمت بیت‌کوین تحت تاثیر تحولات اخیر در پرونده‌...,[]
3,5,23,https://www.tgju.org/news/2638059/آیا-اکنون-وق...,آیا اکنون وقت خرید ریپل است؟,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,ریپل|کمیسیون بورس و اوراق بهادار آمریکا|دادگاه...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از کو...,با انتشار خبری مبنی بر صدور یک حکم مطلوب برای ...,[]
4,30,50,https://www.tgju.org/news/2638164/تحلیل-و-پیش-...,تحلیل و پیش‌بینی قیمت ریپل؛ 25 تیر,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,بیت‌کوین|ریپل|ارزهای دیجیتال|بلک راک|فیدلیتی|ص...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از کو...,پس از پیروزی نسبی ریپل در دادگاه،‌ قیمت آن به ...,[]
...,...,...,...,...,...,...,...,...,...,...,...
1592,39106,1296774,https://www.tgju.org/news/2627237/آیا-طلا-دوبا...,آیا طلا دوباره به 1900 دلار می‌رسد؟,معصومه دانش,اخبار طلا و سکه,یکشنبه 7 خرداد 1402,بهای طلا|خریداران طلا|دلار آمریکا|نرخ بهره|افز...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از «ا...,در بحبوحه‌ی انتظارات برای افزایش 25 واحد پایه‌...,[]
1593,39109,2172897,https://www.tgju.org/news/2567136/آنجا-که-همه-...,آنجا که همه طلا بخواهند!,معصومه دانش,اخبار طلا و سکه,چهارشنبه 25 آبان 1401,بحران روسیه و اوکراین|معاملات آتی طلا|دارایی ا...,به گزارششبکه اطلاع‌رسانی طلا و ارزبه نقل از ای...,با بروز نگرانی در مورد رویداد انفجار موشکی در ...,[]
1594,39113,10078821,https://www.tgju.org/news/2391525/از-هر-سه-زن-...,از هر سه زن، یک نفر قصد خرید ارز دیجیتال دارد,گروه تولید محتوا,اخبار ارزهای دیجیتال,جمعه 13 اسفند 1400,زنان|ارزدیجیتال|تحلیل بنیادین|اخبار ویژه,به گزارش شبکه اطلاع رسانی طلا، سکه و ارز و به ...,بلاک‌فای با آماری که از مقاله مروری خود گرفت ا...,[]
1595,39137,10990361,https://www.tgju.org/news/2579666/پیش-بینی-واق...,پیش‌بینی واقع‌بینانه‌ی قیمت طلا,معصومه دانش,اخبار طلا و سکه,شنبه 26 آذر 1401,فلزات گرانبها|گزارش تورم|فدرال رزرو|چشم‌انداز ...,به گزارششبکه اطلاع‌رسانی طلا و ارز، سایت «اف‌ا...,از آنجا که سرمایه‌گذاران به طور پیوسته در حال ...,[]


In [94]:
# !pip install hazm


In [95]:
import copy
alphabet = set()

for _, row in df.iterrows():
    text = row['text']
    for char in text:
        alphabet.add(char)
ok_alphabet = ' ' 'ابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی' 'ۀءآأؤإئة' '\u200c'
# '0123456789' 'abcdefghijklmnopqrstuvwxyz' 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' '.?!,،:;' '#'

other_alphabet = copy.copy(alphabet)

for char in ok_alphabet:
    if char in other_alphabet:
        other_alphabet.remove(char)

In [96]:
import hazm
import re

stop_words = hazm.stopwords_list()
normalizer = hazm.Normalizer()
lemmatizer = hazm.Lemmatizer()

# mappings
# EXTENDED ARABIC-INDIC DIGIT
mappings1 = [('۰', '0'), ('۱', '1'), ('۲', '2'), ('۳', '3'), ('۴', '4'), ('۵', '5'), ('۶', '6'), ('۷', '7'), ('۸', '8'), ('۹', '9')]

# ARABIC-INDIC DIGIT
mappings2 = [('٠', '0'), ('١', '1'), ('٢', '2'), ('٣', '3'), ('٤', '4'), ('٥', '5'), ('٦', '6'), ('٧', '7'), ('٨', '8'), ('٩', '9')]

# Equivalent letters
mappings3 = [
    ('ە', 'ه'), # ARABIC LETTER AE, ARABIC LETTER HEH
    ('ہ', 'ه'), # ARABIC LETTER HEH GOAL, ARABIC LETTER HEH
    ('ٸ', 'ی'), # ARABIC LETTER HIGH HAMZA YEH, ARABIC LETTER FARSI YEH
    ('ھ', 'ه'), # ARABIC LETTER HEH DOACHASHMEE, ARABIC LETTER HEH
    ('ى', 'ی'), # ARABIC LETTER ALEF MAKSURA, ARABIC LETTER FARSI YEH
    ('ں', 'ن'), # ARABIC LETTER NOON GHUNNA, ARABIC LETTER NOON
    ('ے', 'ی'), # ARABIC LETTER YEH BARREE, ARABIC LETTER FARSI YEH
]
def preprocess(text, mappings):
    # replace with mappings
    for x, y in mappings:
        text = text.replace(x, y)

    # replace specific phrases
    text = text.replace('بیشتر بخوانید', '')
    text = text.replace('به گزارش', '')
    text = text.replace('به نقل از', '')

    # remove text inside parantheses or brackets
    text = re.sub(r'\([^)]*\)|\[[^\]]*\]', '', text)

    # keep words longer than 2
    isnum = re.compile('^\d+$')
    text = " ".join([t for t in hazm.word_tokenize(text) if (len(t) > 2) or (re.match(isnum, t))])

    # replace numbers and English chars with #
    # text = re.sub(r'[a-zA-Z0-9]+', '#', text)
    # replace numbers and English chars with none
    text = re.sub(r'[a-zA-Z0-9]+', '', text)

    # normalize
    text = normalizer.normalize(text)

    # filter out characters not in ok_alphabet
    text = "".join([t for t in text if t in ok_alphabet])

    # filter out stop words
    text = " ".join([t for t in hazm.word_tokenize(text) if t not in stop_words]) # .split() didn't properly remove stopwords

    # stem/lemmatize
    text = " ".join([lemmatizer.lemmatize(t).split('#')[0] for t in hazm.word_tokenize(text)])

    return text
df['text'] = df['text'].apply(lambda text: preprocess(text=text, mappings=mappings1+mappings2+mappings3))

In [97]:
from hazm import word_tokenize
from collections import Counter

def extract_keywords(text, top_n=5):
    words = word_tokenize(text)
    
    word_frequency = Counter(words)
    
    # کلمات با تکرار بیشتر به ترتیب کاهشی جستجو می‌شوند
    keywords = [word for word, _ in word_frequency.most_common(top_n)]
    
    return keywords

# در اینجا df یک DataFrame شماست که در آن ستون 'text' متن‌های پیش‌پردازش شده قرار دارند
# شما می‌توانید نام دیگری که برای DataFrame خود استفاده کرده‌اید را جایگزین کنید
# مثلا: data['text'] یا ... و غیره
df['keywords'] = df['text'].apply(lambda text: extract_keywords(text=text, top_n=5))
df


,id,page_id,url,head,author,category,date,tags,text,summary,hashtags,keywords
0,32298,2172793,https://www.tgju.org/news/2630723/سناریوی-کاهش...,سناریوی کاهش نفت خام، ادامه‌دار,معصومه دانش,اخبار طلا و سکه,دوشنبه 22 خرداد 1402,بهای نفت|نفت خام|نفت برنت|نرخ بهره|افزایش نرخ ...,شبکه اطلاع‌رسانی طلا ارز رویترز قیمتنفتروز دوش...,از آنجا که نگرانی در مورد موضوعاتی همچون تقاضا...,[],"[نفت, کاهش, دلار, بشکه, افزایش]"
1,3,19,https://www.tgju.org/news/2638096/2-49,خرس‌های طلا در کمین!,معصومه دانش,اخبار طلا و سکه,یکشنبه 25 تیر 1402,بهای طلا|بهای نقره|بازار طلا|نرخ بهره|افزایش ن...,شبکه اطلاع‌رسانی طلا ارز کیتکونیوز تورم ضعیف ه...,گرچه طلا قیمت‌های طلا در هفته‌های اخیر رشد داش...,[],"[نقره, طلا, فدرال, رزرو, بازار]"
2,4,21,https://www.tgju.org/news/2638081/چشم-انداز-بی...,چشم‌انداز بیت کوین تا پایان ماه جولای,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,ریپل|بیت کوین|اتریوم|ارزهای دیجیتال|اصلاح قیمت...,شبکه اطلاع‌رسانی طلا ارز بیت‌کوین‌دات‌کام گذشت...,قیمت بیت‌کوین تحت تاثیر تحولات اخیر در پرونده‌...,[],"[سطح, قیمت, کوین, دلار, ماه]"
3,5,23,https://www.tgju.org/news/2638059/آیا-اکنون-وق...,آیا اکنون وقت خرید ریپل است؟,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,ریپل|کمیسیون بورس و اوراق بهادار آمریکا|دادگاه...,شبکه اطلاع‌رسانی طلا ارز کوین‌پدیا کمیسیون بور...,با انتشار خبری مبنی بر صدور یک حکم مطلوب برای ...,[],"[ریپل, قیمت, دلار, سطح, اوراق]"
4,30,50,https://www.tgju.org/news/2638164/تحلیل-و-پیش-...,تحلیل و پیش‌بینی قیمت ریپل؛ 25 تیر,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,بیت‌کوین|ریپل|ارزهای دیجیتال|بلک راک|فیدلیتی|ص...,شبکه اطلاع‌رسانی طلا ارز کوین‌پدیا قیمتبیت کوی...,پس از پیروزی نسبی ریپل در دادگاه،‌ قیمت آن به ...,[],"[ریپل, قیمت, کوین, بازار, افزایش]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1592,39106,1296774,https://www.tgju.org/news/2627237/آیا-طلا-دوبا...,آیا طلا دوباره به 1900 دلار می‌رسد؟,معصومه دانش,اخبار طلا و سکه,یکشنبه 7 خرداد 1402,بهای طلا|خریداران طلا|دلار آمریکا|نرخ بهره|افز...,شبکه اطلاع‌رسانی طلا ارز اف‌ایکس استریت قیمتطل...,در بحبوحه‌ی انتظارات برای افزایش 25 واحد پایه‌...,[],"[دلار, طلا, قیمت, ایالات‌متحده, فدرال]"
1593,39109,2172897,https://www.tgju.org/news/2567136/آنجا-که-همه-...,آنجا که همه طلا بخواهند!,معصومه دانش,اخبار طلا و سکه,چهارشنبه 25 آبان 1401,بحران روسیه و اوکراین|معاملات آتی طلا|دارایی ا...,شبکه اطلاع‌رسانی طلا ارز اینوستینگ قیمتطلاروز ...,با بروز نگرانی در مورد رویداد انفجار موشکی در ...,[],"[افزایش, کاهش, گذشته, طلا, بالا]"
1594,39113,10078821,https://www.tgju.org/news/2391525/از-هر-سه-زن-...,از هر سه زن، یک نفر قصد خرید ارز دیجیتال دارد,گروه تولید محتوا,اخبار ارزهای دیجیتال,جمعه 13 اسفند 1400,زنان|ارزدیجیتال|تحلیل بنیادین|اخبار ویژه,شبکه اطلاع‌رسانی طلا سکه ارز دیلی‌کوین درسال ن...,بلاک‌فای با آماری که از مقاله مروری خود گرفت ا...,[],"[ارز, دیجیتال, درصد, زن, نفر]"
1595,39137,10990361,https://www.tgju.org/news/2579666/پیش-بینی-واق...,پیش‌بینی واقع‌بینانه‌ی قیمت طلا,معصومه دانش,اخبار طلا و سکه,شنبه 26 آذر 1401,فلزات گرانبها|گزارش تورم|فدرال رزرو|چشم‌انداز ...,شبکه اطلاع‌رسانی طلا ارز سایت اف‌ایکس استریت م...,از آنجا که سرمایه‌گذاران به طور پیوسته در حال ...,[],"[طلا, درصد, دلار, افزایش, قیمت]"


In [98]:
# نمایش کلمات کلیدی هر متن
for index, row in df.iterrows():
    print(f"متن {index + 1} - کلمات کلیدی:")
    print(row['keywords'])
    print()


متن 1 - کلمات کلیدی:
['نفت', 'کاهش', 'دلار', 'بشکه', 'افزایش']

متن 2 - کلمات کلیدی:
['نقره', 'طلا', 'فدرال', 'رزرو', 'بازار']

متن 3 - کلمات کلیدی:
['سطح', 'قیمت', 'کوین', 'دلار', 'ماه']

متن 4 - کلمات کلیدی:
['ریپل', 'قیمت', 'دلار', 'سطح', 'اوراق']

متن 5 - کلمات کلیدی:
['ریپل', 'قیمت', 'کوین', 'بازار', 'افزایش']

متن 6 - کلمات کلیدی:
['طلا', 'درصد', 'هفته', 'کاهش', 'دلار']

متن 7 - کلمات کلیدی:
['سطح', 'یورو', 'دلار', 'فدرال', 'رزرو']

متن 8 - کلمات کلیدی:
['دلار', 'کاهش', 'افزایش', 'درصد', 'سطح']

متن 9 - کلمات کلیدی:
['ریپل', 'قیمت', 'نزول', 'کاهش', 'حرکت']

متن 10 - کلمات کلیدی:
['قیمت', 'بستهشدن', 'نوسان', 'مقادیر', 'قبل']

متن 11 - کلمات کلیدی:
['دلار', 'نقره', 'فشار', 'صعود', 'دلیل']

متن 12 - کلمات کلیدی:
['کوین', 'سولانا', 'درصد', 'بازار', 'ریپل']

متن 13 - کلمات کلیدی:
['نفت', 'دلار', 'کاهش', 'بشکه', 'افزایش']

متن 14 - کلمات کلیدی:
['اوکراین', 'دوج\u200cکوین', 'ارز', 'مارکوس', 'قیمت']

متن 15 - کلمات کلیدی:
['کوین', 'بیت', 'درصد', 'بازار', 'کیوساکی']

متن 16 - کلمات کلیدی:

In [99]:
df['tags'] = df['tags'].apply(lambda x: x.split('|') if not pd.isnull(x) else [])
df

,id,page_id,url,head,author,category,date,tags,text,summary,hashtags,keywords
0,32298,2172793,https://www.tgju.org/news/2630723/سناریوی-کاهش...,سناریوی کاهش نفت خام، ادامه‌دار,معصومه دانش,اخبار طلا و سکه,دوشنبه 22 خرداد 1402,"[بهای نفت, نفت خام, نفت برنت, نرخ بهره, افزایش...",شبکه اطلاع‌رسانی طلا ارز رویترز قیمتنفتروز دوش...,از آنجا که نگرانی در مورد موضوعاتی همچون تقاضا...,[],"[نفت, کاهش, دلار, بشکه, افزایش]"
1,3,19,https://www.tgju.org/news/2638096/2-49,خرس‌های طلا در کمین!,معصومه دانش,اخبار طلا و سکه,یکشنبه 25 تیر 1402,"[بهای طلا, بهای نقره, بازار طلا, نرخ بهره, افز...",شبکه اطلاع‌رسانی طلا ارز کیتکونیوز تورم ضعیف ه...,گرچه طلا قیمت‌های طلا در هفته‌های اخیر رشد داش...,[],"[نقره, طلا, فدرال, رزرو, بازار]"
2,4,21,https://www.tgju.org/news/2638081/چشم-انداز-بی...,چشم‌انداز بیت کوین تا پایان ماه جولای,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,"[ریپل, بیت کوین, اتریوم, ارزهای دیجیتال, اصلاح...",شبکه اطلاع‌رسانی طلا ارز بیت‌کوین‌دات‌کام گذشت...,قیمت بیت‌کوین تحت تاثیر تحولات اخیر در پرونده‌...,[],"[سطح, قیمت, کوین, دلار, ماه]"
3,5,23,https://www.tgju.org/news/2638059/آیا-اکنون-وق...,آیا اکنون وقت خرید ریپل است؟,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,"[ریپل, کمیسیون بورس و اوراق بهادار آمریکا, داد...",شبکه اطلاع‌رسانی طلا ارز کوین‌پدیا کمیسیون بور...,با انتشار خبری مبنی بر صدور یک حکم مطلوب برای ...,[],"[ریپل, قیمت, دلار, سطح, اوراق]"
4,30,50,https://www.tgju.org/news/2638164/تحلیل-و-پیش-...,تحلیل و پیش‌بینی قیمت ریپل؛ 25 تیر,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,"[بیت‌کوین, ریپل, ارزهای دیجیتال, بلک راک, فیدل...",شبکه اطلاع‌رسانی طلا ارز کوین‌پدیا قیمتبیت کوی...,پس از پیروزی نسبی ریپل در دادگاه،‌ قیمت آن به ...,[],"[ریپل, قیمت, کوین, بازار, افزایش]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1592,39106,1296774,https://www.tgju.org/news/2627237/آیا-طلا-دوبا...,آیا طلا دوباره به 1900 دلار می‌رسد؟,معصومه دانش,اخبار طلا و سکه,یکشنبه 7 خرداد 1402,"[بهای طلا, خریداران طلا, دلار آمریکا, نرخ بهره...",شبکه اطلاع‌رسانی طلا ارز اف‌ایکس استریت قیمتطل...,در بحبوحه‌ی انتظارات برای افزایش 25 واحد پایه‌...,[],"[دلار, طلا, قیمت, ایالات‌متحده, فدرال]"
1593,39109,2172897,https://www.tgju.org/news/2567136/آنجا-که-همه-...,آنجا که همه طلا بخواهند!,معصومه دانش,اخبار طلا و سکه,چهارشنبه 25 آبان 1401,"[بحران روسیه و اوکراین, معاملات آتی طلا, دارای...",شبکه اطلاع‌رسانی طلا ارز اینوستینگ قیمتطلاروز ...,با بروز نگرانی در مورد رویداد انفجار موشکی در ...,[],"[افزایش, کاهش, گذشته, طلا, بالا]"
1594,39113,10078821,https://www.tgju.org/news/2391525/از-هر-سه-زن-...,از هر سه زن، یک نفر قصد خرید ارز دیجیتال دارد,گروه تولید محتوا,اخبار ارزهای دیجیتال,جمعه 13 اسفند 1400,"[زنان, ارزدیجیتال, تحلیل بنیادین, اخبار ویژه]",شبکه اطلاع‌رسانی طلا سکه ارز دیلی‌کوین درسال ن...,بلاک‌فای با آماری که از مقاله مروری خود گرفت ا...,[],"[ارز, دیجیتال, درصد, زن, نفر]"
1595,39137,10990361,https://www.tgju.org/news/2579666/پیش-بینی-واق...,پیش‌بینی واقع‌بینانه‌ی قیمت طلا,معصومه دانش,اخبار طلا و سکه,شنبه 26 آذر 1401,"[فلزات گرانبها, گزارش تورم, فدرال رزرو, چشم‌ان...",شبکه اطلاع‌رسانی طلا ارز سایت اف‌ایکس استریت م...,از آنجا که سرمایه‌گذاران به طور پیوسته در حال ...,[],"[طلا, درصد, دلار, افزایش, قیمت]"


In [100]:
import pandas as pd

def is_empty_hashtags(cell_value):
    return len(cell_value) == 0

for index, row in df.iterrows():
    if is_empty_hashtags(row['hashtags']):
        if not is_empty_hashtags(row['tags']):
            df.at[index, 'hashtags'] = row['tags']
        else:
            df.at[index, 'hashtags'] = row['keywords']

df

,id,page_id,url,head,author,category,date,tags,text,summary,hashtags,keywords
0,32298,2172793,https://www.tgju.org/news/2630723/سناریوی-کاهش...,سناریوی کاهش نفت خام، ادامه‌دار,معصومه دانش,اخبار طلا و سکه,دوشنبه 22 خرداد 1402,"[بهای نفت, نفت خام, نفت برنت, نرخ بهره, افزایش...",شبکه اطلاع‌رسانی طلا ارز رویترز قیمتنفتروز دوش...,از آنجا که نگرانی در مورد موضوعاتی همچون تقاضا...,"[بهای نفت, نفت خام, نفت برنت, نرخ بهره, افزایش...","[نفت, کاهش, دلار, بشکه, افزایش]"
1,3,19,https://www.tgju.org/news/2638096/2-49,خرس‌های طلا در کمین!,معصومه دانش,اخبار طلا و سکه,یکشنبه 25 تیر 1402,"[بهای طلا, بهای نقره, بازار طلا, نرخ بهره, افز...",شبکه اطلاع‌رسانی طلا ارز کیتکونیوز تورم ضعیف ه...,گرچه طلا قیمت‌های طلا در هفته‌های اخیر رشد داش...,"[بهای طلا, بهای نقره, بازار طلا, نرخ بهره, افز...","[نقره, طلا, فدرال, رزرو, بازار]"
2,4,21,https://www.tgju.org/news/2638081/چشم-انداز-بی...,چشم‌انداز بیت کوین تا پایان ماه جولای,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,"[ریپل, بیت کوین, اتریوم, ارزهای دیجیتال, اصلاح...",شبکه اطلاع‌رسانی طلا ارز بیت‌کوین‌دات‌کام گذشت...,قیمت بیت‌کوین تحت تاثیر تحولات اخیر در پرونده‌...,"[ریپل, بیت کوین, اتریوم, ارزهای دیجیتال, اصلاح...","[سطح, قیمت, کوین, دلار, ماه]"
3,5,23,https://www.tgju.org/news/2638059/آیا-اکنون-وق...,آیا اکنون وقت خرید ریپل است؟,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,"[ریپل, کمیسیون بورس و اوراق بهادار آمریکا, داد...",شبکه اطلاع‌رسانی طلا ارز کوین‌پدیا کمیسیون بور...,با انتشار خبری مبنی بر صدور یک حکم مطلوب برای ...,"[ریپل, کمیسیون بورس و اوراق بهادار آمریکا, داد...","[ریپل, قیمت, دلار, سطح, اوراق]"
4,30,50,https://www.tgju.org/news/2638164/تحلیل-و-پیش-...,تحلیل و پیش‌بینی قیمت ریپل؛ 25 تیر,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,یکشنبه 25 تیر 1402,"[بیت‌کوین, ریپل, ارزهای دیجیتال, بلک راک, فیدل...",شبکه اطلاع‌رسانی طلا ارز کوین‌پدیا قیمتبیت کوی...,پس از پیروزی نسبی ریپل در دادگاه،‌ قیمت آن به ...,"[بیت‌کوین, ریپل, ارزهای دیجیتال, بلک راک, فیدل...","[ریپل, قیمت, کوین, بازار, افزایش]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1592,39106,1296774,https://www.tgju.org/news/2627237/آیا-طلا-دوبا...,آیا طلا دوباره به 1900 دلار می‌رسد؟,معصومه دانش,اخبار طلا و سکه,یکشنبه 7 خرداد 1402,"[بهای طلا, خریداران طلا, دلار آمریکا, نرخ بهره...",شبکه اطلاع‌رسانی طلا ارز اف‌ایکس استریت قیمتطل...,در بحبوحه‌ی انتظارات برای افزایش 25 واحد پایه‌...,"[بهای طلا, خریداران طلا, دلار آمریکا, نرخ بهره...","[دلار, طلا, قیمت, ایالات‌متحده, فدرال]"
1593,39109,2172897,https://www.tgju.org/news/2567136/آنجا-که-همه-...,آنجا که همه طلا بخواهند!,معصومه دانش,اخبار طلا و سکه,چهارشنبه 25 آبان 1401,"[بحران روسیه و اوکراین, معاملات آتی طلا, دارای...",شبکه اطلاع‌رسانی طلا ارز اینوستینگ قیمتطلاروز ...,با بروز نگرانی در مورد رویداد انفجار موشکی در ...,"[بحران روسیه و اوکراین, معاملات آتی طلا, دارای...","[افزایش, کاهش, گذشته, طلا, بالا]"
1594,39113,10078821,https://www.tgju.org/news/2391525/از-هر-سه-زن-...,از هر سه زن، یک نفر قصد خرید ارز دیجیتال دارد,گروه تولید محتوا,اخبار ارزهای دیجیتال,جمعه 13 اسفند 1400,"[زنان, ارزدیجیتال, تحلیل بنیادین, اخبار ویژه]",شبکه اطلاع‌رسانی طلا سکه ارز دیلی‌کوین درسال ن...,بلاک‌فای با آماری که از مقاله مروری خود گرفت ا...,"[زنان, ارزدیجیتال, تحلیل بنیادین, اخبار ویژه]","[ارز, دیجیتال, درصد, زن, نفر]"
1595,39137,10990361,https://www.tgju.org/news/2579666/پیش-بینی-واق...,پیش‌بینی واقع‌بینانه‌ی قیمت طلا,معصومه دانش,اخبار طلا و سکه,شنبه 26 آذر 1401,"[فلزات گرانبها, گزارش تورم, فدرال رزرو, چشم‌ان...",شبکه اطلاع‌رسانی طلا ارز سایت اف‌ایکس استریت م...,از آنجا که سرمایه‌گذاران به طور پیوسته در حال ...,"[فلزات گرانبها, گزارش تورم, فدرال رزرو, چشم‌ان...","[طلا, درصد, دلار, افزایش, قیمت]"


In [101]:
# df.to_csv(csv_file_path + 'export.csv')

In [102]:
from kafka import KafkaProducer
import os, random
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql.functions import mean, col, desc, count, isnull, isnan, when, rank, sum
import pandas as pd
import matplotlib.pyplot as plt


producer = KafkaProducer(bootstrap_servers='127.0.0.1:9092')

rows = df.apply(lambda row: '\t'.join([str(value) for value in row]), axis=1)

for row in rows:
    producer.send('news_from_step2', value=row.encode('utf-8'))

producer.close()

print(f'Sent {len(rows)} lines in total.')


Sent 1597 lines in total.


# MONGOOO

In [103]:
# !pip install pymongo

In [104]:
# from pymongo import MongoClient

In [105]:
# mongo_url = "localhost:27017"
# client = MongoClient(mongo_url)
# db = client['BigDataFinalProject']
# collection = db['News_Text']
# # Replace 'data_to_insert' with the dictionary or data you want to insert into the collection
# data_to_insert = {
#     'name': 'John Doe',
#     'age': 30,
#     'email': 'johndoe@example.com'
# }

# # Insert the data into the collection
# inserted_data = collection.insert_one(data_to_insert)

# # Optionally, you can print the inserted document's ID
# print("Inserted document ID:", inserted_data.inserted_id)